# Suchoptionen Metadaten

In [1]:
import pandas as pd

"""Einlesen der Metadaten-Tabelle als Dataframe"""
df_metadata = pd.read_csv("files\metadata_letters.csv")

def list_content(column_name):
    """Auflisten der unique-Werte einer Spalte des Dataframe in alphabetischer Reihenfolge"""
    names = (df_metadata[column_name].unique())
    for i in sorted(names):
        print(i)

def print_result(column_name, name):
    """Ausgabe der Zeilen des Dataframe, die ein Suchwort (= name) enthalten"""
    result = df_metadata.loc[df_metadata[column_name] == name, "Dateiname"].to_list()
    for i in result:
        print(i)

## Suche nach Absender*innen

### Ausgeben einer Liste von Absender*innen

In [2]:
list_content("Absender*in")

Aapeli Saarisalo
Benno Landsberger
Cyril John Gadd
François Thureau-Dangin
Friedrich Stummer
Giorgio Castellino
Harri Holma
Jakob J. Finkelstein
Joh. F. Kunstmann
Johannes Pedersen
Karl Budde
Karl Friedrich Müller
Reginald Campbell Thompson
Richard Hartmann
Walter Andrae
Walter G. Kunstmann


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [3]:
#Einfügen des gesuchten Namens
name = "Walter Andrae"

print_result("Absender*in", name)

1932-06-30_Andrae_Landsberger


## Suche nach Empfänger*innen

### Ausgeben einer Liste von Empfänger*innen

In [4]:
list_content("Empfänger*in")

Benno Landsberger
François Thureau-Dangin
Jussi Aro
Karl Friedrich Müller
Torgny Segerstedt


### Auswahl eines Namens und Ausgabe der zugehörigen Briefe

In [5]:
#Einfügen des gesuchten Namens
name = "Jussi Aro"

print_result("Empfänger*in", name)

1960-09-27_Landsberger_Aro


## Suche nach Aufenthaltsorten der Absender*innen

### Ausgeben einer Liste von Aufenthaltsorten

In [6]:
list_content("Aufenthaltsort Absender*in")

Ankara
Berkeley
Berlin
Chicago
Frydek
Heidelberg
Helsinki 
Kopenhagen
Leipzig
London
Marburg
Oxford
Paris
Porto Alegre
Rom
Wiederitzsch 
Würzburg


### Auswahl eines Ortes und Ausgabe der zugehörigen Briefe

In [7]:
#Einfügen des gesuchten Ortes
name = "Leipzig"

print_result("Aufenthaltsort Absender*in", name)

1929-07-10_Mueller_Landsberger


## Suche nach einem Datumszeitraum

In [8]:
#Einfügen eines Datumszeitraums
start_date = "1929-01-01"
end_date = "1931-01-01"

"""Umwandeln des Datums in ein Datetime-Format; Definieren des Datumszeitraums"""
df_metadata["Datum (ISO 8601)"] = pd.to_datetime(df_metadata["Datum (ISO 8601)"], format="ISO8601")
mask = (df_metadata["Datum (ISO 8601)"] > start_date) & (df_metadata["Datum (ISO 8601)"] <= end_date)

result_date = df_metadata.loc[mask, "Dateiname"].to_list()
for i in result_date:
    print(i)

1929-06-06_Landsberger_ThureauDangin
1929-07-10_Mueller_Landsberger
1929-12-09_Hartmann_Landsberger
1930-01-03_Budde_Landsberger


## Suche nach der Briefsprache

### Ausgeben einer Liste von Sprachen

In [9]:
list_content("Sprache")

deutsch
englisch
französisch


### Auswahl einer Sprache und der zugehörigen Briefe

In [10]:
#Einfügen der gesuchten Sprache
name = "englisch"

print_result("Sprache", name)

1933-03-08_Gadd_Landsberger
1943-10-01_Thompson_Landsberger
1956-02-15_Landsberger_Segerstedt
1960-09-27_Landsberger_Aro
1962-02-19_Finkelstein_Landsberger


# Suchoptionen Briefinhalte

In [11]:
import glob
import re
from bs4 import BeautifulSoup
from os.path import join

def content_search(tag, attribute):
    """Einlesen der in TEI kodierten Textinhalte;
    Auffinden von ausgesuchten Tags und Attributen;
    Ausgabe des Ergebnis im Format 'Gesuchtes Tag: Briefname'
    """
    for file in glob.glob(join(f"files", "letters", "tei", "*.xml")):
        with open(file, "r", encoding="utf-8") as infile:
            text = infile.read()
            filename = file.split("\\")[-1].split(".")[0]
            soup = BeautifulSoup(text, "xml")
            tag_list = soup.find_all(tag, attribute)
            tag_tuple_list = []
            for i in tag_list:
                i = i.text
                i = re.sub("-\n", "", i) #Zusammenführung von Wörtern, die am Zeilenende durch Bindestrich getrennt werden
                i = re.sub("\n", " ", i) #Zusammenführung von Bezeichnungen, die aus mehreren Wörtern bestehen und auf zwei oder mehr Zeilen aufgeteilt sind
                i = re.sub("  ", "", i)  #Entfernen von doppelten Leerzeichen
                i = (i, filename)
                tag_tuple_list.append(i)
            tag_tuple_list = list(set(tag_tuple_list))
            tag_tuple_list.sort(key=lambda t : tuple(t[0].lower()))
            for x,y in tag_tuple_list:
                print(x + ": " + y)

### Erwähnungen von modernen Personen

In [12]:
content_search("persName", {"type" : "modern"})

G.: 1929-12-09_Hartmann_Landsberger
Götze: 1929-12-09_Hartmann_Landsberger
Schott: 1929-12-09_Hartmann_Landsberger
Gadd: 1931-03-11_ThureauDangin_Landsberger
Langdon: 1931-03-11_ThureauDangin_Landsberger
Zimmern: 1931-03-11_ThureauDangin_Landsberger
Ronzevalle: 1932-03-22_Pedersen_Landsberger
Zimmern: 1932-03-22_Pedersen_Landsberger
Zimmerns: 1932-03-22_Pedersen_Landsberger
Ehelolf: 1932-06-30_Andrae_Landsberger
Hehn: 1933-04-17_Stummer_Landsberger
Hehns: 1933-04-17_Stummer_Landsberger
Johannes Hehn: 1933-04-17_Stummer_Landsberger
Gadd: 1936-05-05_Landsberger_Mueller
Schuster: 1936-05-05_Landsberger_Mueller
Friedrich: 1937-01-29_Mueller_Landsberger
Peiserschen: 1937-01-29_Mueller_Landsberger
Pohl: 1937-01-29_Mueller_Landsberger
Schuster: 1937-01-29_Mueller_Landsberger
Zimmernsche: 1937-01-29_Mueller_Landsberger
Zimmernschen: 1937-01-29_Mueller_Landsberger
Frankfort: 1937-02-09_Landsberger_Mueller
Koschaker: 1937-02-09_Landsberger_Mueller
Langdon: 1937-02-09_Landsberger_Mueller
Moortgat

### Erwähnungen von Konferenzen/Kongressen

In [13]:
content_search("title", {"type" : "meeting"})

Orientalisten Congress: 1937-04-12_Castellino_Landsberger


### Erwähnungen von modernen Orten

In [14]:
content_search("placeName", {"type" : "modern"})

Leipzig: 1929-12-09_Hartmann_Landsberger
Marburg: 1929-12-09_Hartmann_Landsberger
Leipzig: 1931-03-11_ThureauDangin_Landsberger
Deutschland: 1932-03-22_Pedersen_Landsberger
Hama: 1932-03-22_Pedersen_Landsberger
Israel: 1932-03-22_Pedersen_Landsberger
Leiden: 1932-03-22_Pedersen_Landsberger
Leipzig: 1932-03-22_Pedersen_Landsberger
London: 1932-03-22_Pedersen_Landsberger
Deutschland: 1936-05-05_Landsberger_Mueller
England: 1936-05-05_Landsberger_Mueller
London: 1936-05-05_Landsberger_Mueller
Oesterreich: 1936-05-05_Landsberger_Mueller
Polen: 1936-05-05_Landsberger_Mueller
Tchechoslowakei: 1936-05-05_Landsberger_Mueller
Londoner: 1937-02-09_Landsberger_Mueller
Rom: 1937-04-12_Castellino_Landsberger


### Erwähnungen von modernen Institutionen

In [15]:
content_search("orgName", {"type" : "academy"})

Sächs. Akademie: 1932-06-30_Andrae_Landsberger


In [16]:
content_search("orgName", {"type" : "institute"})

Orientalischen und am Sprachwissenschaftlichen (indogermanistischen) Seminar: 1929-12-09_Hartmann_Landsberger


### Erwähnungen von altorientalischen Orten

In [17]:
content_search("placeName", {"type" : "ANE"})

Assur: 1929-07-10_Mueller_Landsberger
assyrischen Reiches: 1929-07-10_Mueller_Landsberger
Ras Shamra: 1931-03-11_ThureauDangin_Landsberger
Babylon: 1937-01-29_Mueller_Landsberger
Niniveh: 1937-01-29_Mueller_Landsberger


### Erwähnungen von altorientalischen Göttern

In [18]:
content_search("name", {"type" : "deity"})

Assur: 1929-07-10_Mueller_Landsberger
Assurs: 1929-07-10_Mueller_Landsberger
Enlil: 1929-07-10_Mueller_Landsberger
Marduk: 1929-07-10_Mueller_Landsberger
Māratnāri: 1929-07-10_Mueller_Landsberger
Ninlil: 1929-07-10_Mueller_Landsberger
Serua: 1929-07-10_Mueller_Landsberger
Marat-nari: 1937-01-29_Mueller_Landsberger
Nanâ: 1937-04-12_Castellino_Landsberger


### Erwähnungen von Forschungspublikationen

In [19]:
content_search("title", {"type" : "journal"})

MDOG 60: 1929-07-10_Mueller_Landsberger
RA XIV pp. 1 et suiv: 1931-03-11_ThureauDangin_Landsberger
MFOB: 1932-03-22_Pedersen_Landsberger
ZA: 1933-04-17_Stummer_Landsberger
ZA: 1937-02-09_Landsberger_Mueller


In [20]:
content_search("title", {"type" : "monograph"})

1er volume de la Neue Folge des Leipziger Semitische Studien: 1931-03-11_ThureauDangin_Landsberger
ana ittišu: 1931-03-11_ThureauDangin_Landsberger
Assur-Zoologie: 1932-06-30_Andrae_Landsberger
ana ittisu: 1937-01-29_Mueller_Landsberger
BBR: 1937-01-29_Mueller_Landsberger
Rit.Acc.: 1937-01-29_Mueller_Landsberger
ea: 1937-02-09_Landsberger_Mueller
Zimmern, Neujahrsfest: 1937-02-09_Landsberger_Mueller


### Erwähnungen von Keilschrifttexten

In [21]:
content_search("title", {"type" : "cuneiformtext"})

8. Feldzug Sargons: 1929-07-10_Mueller_Landsberger
KAR 139: 1929-07-10_Mueller_Landsberger
Krönungsritual: 1929-07-10_Mueller_Landsberger
RM. II, 36: 1931-03-11_ThureauDangin_Landsberger
Harra Tafel 1: 1936-05-05_Landsberger_Mueller
215: 1937-01-29_Mueller_Landsberger
K.10209: 1937-01-29_Mueller_Landsberger
K.3438a: 1937-01-29_Mueller_Landsberger
K.3455: 1937-01-29_Mueller_Landsberger
KAR 146: 1937-01-29_Mueller_Landsberger
KAR 217: 1937-01-29_Mueller_Landsberger
K 10209: 1937-02-09_Landsberger_Mueller
K 3438a: 1937-02-09_Landsberger_Mueller
lu=amelu: 1937-02-09_Landsberger_Mueller
"S. Kultlieder" unter N.215 (V.S. X, 215): 1937-04-12_Castellino_Landsberger


### Erwähnungen von altorientalischen Textpassagen oder Lemmata

### Akkadisch

In [22]:
content_search("foreign", {"xml:lang" : "akk"})

ab-ba-sa: 1929-07-10_Mueller_Landsberger
abnu: 1929-07-10_Mueller_Landsberger
ip-pal: 1929-07-10_Mueller_Landsberger
kararu: 1929-07-10_Mueller_Landsberger
Pû-lišānu: 1929-07-10_Mueller_Landsberger
lapātu: 1936-05-05_Landsberger_Mueller
lapātu ša nakāsi: 1936-05-05_Landsberger_Mueller
naru: 1937-01-29_Mueller_Landsberger
takultu: 1937-01-29_Mueller_Landsberger
il-ta-am: 1937-04-12_Castellino_Landsberger


### Erwähnungen von altorientalischen Sprachen

In [23]:
content_search("lang", "")

Aramäischen: 1929-07-10_Mueller_Landsberger
Akkadisch: 1929-12-09_Hartmann_Landsberger
accadienne: 1931-03-11_ThureauDangin_Landsberger
sumérienne: 1931-03-11_ThureauDangin_Landsberger
aramäische: 1932-03-22_Pedersen_Landsberger
babylonischen: 1932-03-22_Pedersen_Landsberger
sumerischen: 1932-03-22_Pedersen_Landsberger
altbabylonischen: 1936-05-05_Landsberger_Mueller
akkadischen: 1937-01-29_Mueller_Landsberger
assyrischer: 1937-01-29_Mueller_Landsberger
assyrischen: 1937-02-09_Landsberger_Mueller
Akkadischen: 1937-04-12_Castellino_Landsberger
Hebr.: 1937-04-12_Castellino_Landsberger
